In [1]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint

In [2]:
training = r'C:\Users\itars\M Pr\archive\real_vs_fake\real-vs-fake\train'
valid = r'C:\Users\itars\M Pr\archive\real_vs_fake\real-vs-fake\valid'
test = r'C:\Users\itars\M Pr\archive\real_vs_fake\real-vs-fake\test'

In [3]:
image_gen = ImageDataGenerator(rescale=1./255.)
batch_size = 64
train_flow = image_gen.flow_from_directory(
    training,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='binary'
)
valid_flow = image_gen.flow_from_directory(
    valid,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='binary'
)

Found 100000 images belonging to 2 classes.
Found 20000 images belonging to 2 classes.


In [4]:
cnn = tf.keras.models.Sequential()

In [5]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[224, 224, 3]))

In [6]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

In [7]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

In [8]:
cnn.add(tf.keras.layers.Flatten())

In [9]:
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

In [10]:
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

In [11]:
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

In [12]:
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

In [13]:
cnn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 111, 111, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 109, 109, 32)      9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 54, 54, 32)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 93312)             0         
                                                                 
 dense (Dense)               (None, 128)               1

In [14]:
opt = tf.keras.optimizers.RMSprop(
        learning_rate=0.0001,
        momentum=0.01)

In [15]:
cnn.compile(optimizer = opt , loss = 'binary_crossentropy', metrics = ['accuracy'])

In [20]:
checkpoint = ModelCheckpoint('weights.h5', save_weights_only=True)

In [21]:
history = None

In [22]:
train_steps = 400//batch_size
valid_steps = 50//batch_size
history = cnn.fit_generator(
    train_flow,
    epochs=1,
    steps_per_epoch = train_steps,
    validation_data = valid_flow,
    validation_steps = valid_steps,
    callbacks=[checkpoint]
)

C:\Users\itars\AppData\Local\Temp\ipykernel_12588\449151198.py:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = cnn.fit_generator(


6/6 [==============================] - 15s 2s/step - loss: 0.0132 - accuracy: 0.9948


In [23]:
cnn.load_weights('weights.h5')

In [19]:
train_steps = 40000//batch_size
valid_steps = 5000//batch_size
history = cnn.fit_generator(
    train_flow,
    epochs=15,
    steps_per_epoch = train_steps,
    validation_data = valid_flow,
    validation_steps = valid_steps,
    callbacks=[checkpoint]
)

C:\Users\itars\AppData\Local\Temp\ipykernel_20868\2188059181.py:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = cnn.fit_generator(


Epoch 1/15
625/625 [==============================] - 1459s 2s/step - loss: 0.2362 - accuracy: 0.9057 - val_loss: 0.2244 - val_accuracy: 0.9125
Epoch 2/15
625/625 [==============================] - 1404s 2s/step - loss: 0.1878 - accuracy: 0.9269 - val_loss: 0.2245 - val_accuracy: 0.9052
Epoch 3/15
625/625 [==============================] - 1322s 2s/step - loss: 0.1525 - accuracy: 0.9430 - val_loss: 0.1776 - val_accuracy: 0.9291
Epoch 4/15
625/625 [==============================] - 1348s 2s/step - loss: 0.1229 - accuracy: 0.9543 - val_loss: 0.1821 - val_accuracy: 0.9275
Epoch 5/15
625/625 [==============================] - 1369s 2s/step - loss: 0.1018 - accuracy: 0.9635 - val_loss: 0.1873 - val_accuracy: 0.9273
Epoch 6/15
625/625 [==============================] - 1344s 2s/step - loss: 0.0769 - accuracy: 0.9725 - val_loss: 0.1448 - val_accuracy: 0.9475
Epoch 7/15
625/625 [==============================] - 1338s 2s/step - loss: 0.0607 - accuracy: 0.9791 - val_loss: 0.1428 - val_accuracy:

In [24]:
latest_epoch_accuracy = history.history['accuracy'][-1]

In [25]:
print("Accuracy of the latest epoch:", latest_epoch_accuracy)

Accuracy of the latest epoch: 0.9947916865348816


In [26]:
image_gen = ImageDataGenerator(rescale=1./255.)
test_flow = image_gen.flow_from_directory(
    test,
    target_size=(224, 224),
    batch_size=1,
    shuffle=False,
    class_mode='binary'
)

Found 20000 images belonging to 2 classes.


In [27]:
results = cnn.evaluate(test_flow)

20000/20000 [==============================] - 299s 15ms/step - loss: 0.1341 - accuracy: 0.9634


In [27]:
import numpy as np
from keras.utils import load_img, img_to_array
test_image=load_img(r"C:\Users\itars\Downloads\fotor_2023-5-13_16_9_28.jpg", target_size = (224, 224))
test_image=img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = cnn.predict(test_image)
train_flow.class_indices
if result[0][0] == 1:
  prediction = 'REAL'
else:
  prediction = 'FAKE'

1/1 [==============================] - 0s 196ms/step


In [28]:
print(prediction)

REAL
